Created on 20 Oct 2020
DJKC 워크숍용

ㅇ과제: 포털 네이버에서 가장 많이 본 기사

@author: Osung

In [1]:
from selenium import webdriver
from selenium.common.exceptions import UnexpectedAlertPresentException 
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime as dt

In [2]:
browser = webdriver.Chrome() #코딩으로 콘트롤 하는 웹브라우저

In [3]:
# html 파싱 함수 정의 
def parser(url):
    try: 
        browser.get(url)
    except UnexpectedAlertPresentException:  #될 때가 있고 안 될 때가 있음. 예외적인 경우를 위한 코드
        print("alert")
        time.sleep(1)
        alert = browser.switch_to.alert
        time.sleep(1)
        alert.accept()
        time.sleep(1)
        WebDriverWait(browser, 5)
    HTML = browser.execute_script("return document.body.innerHTML")
    return BeautifulSoup(HTML, 'html.parser')

In [9]:
# 파싱된 내용에서 필요한 부분 가져오는 함수
def get_data(parsed):
    heads = parsed.select('.ranking_headline')
    
    heads_txt = list()
    for item in heads:
        heads_txt.append(item.a.text.strip())
    
    links = list()
    for item in heads:
        links.append(item.a.attrs['href'])
    
    ledes = parsed.select('.ranking_lede')
    ledes_txt = list()
    for item in ledes:
        ledes_txt.append(item.text.strip())
        
    offices = parsed.select('.ranking_office')
    offices_txt = list()
    for item in offices:
        offices_txt.append(item.text.strip())
        
    views = parsed.select('.ranking_view')
    views_num = list()
    for item in views:
        views_num.append(int(''.join(filter(str.isdigit,item.text.strip()))))

    df = pd.DataFrame(columns = list(['제목', '링크', '리드', '언론사', '조회수']))
    df = df.assign(제목=heads_txt, 링크=links, 리드=ledes_txt, 언론사= offices_txt, 조회수=views_num)
    return df

In [10]:
#사전 정의 변수
sectionIDs = ('100', '101', '102', '103', '104', '105') #고정된 섹션값
start_date = dt.date(2019, 1, 1) #크롤링 시작일
r = 31 #몇일동안 할 것인가 범위

현재 작업 중인 폴더 아래에 연도 폴더 먼저 만들어주기(깔끔한 저장용)

In [12]:
#크롤링!
for i in range(r):
    date = start_date+dt.timedelta(days=i)
    date_str = date.strftime("%Y%m%d")
    for sectionID in sectionIDs:
        url = 'https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&sectionId='\
            +sectionID+'&date='+date_str
        parsed = parser(url)
        df = get_data(parsed)
        df.to_csv(date_str[:4]+'\\'+date_str+sectionID+".csv" ,encoding='utf-8-sig')